<a href="https://colab.research.google.com/github/jchanke/dlsys-hw4/blob/main/hw4_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-714 Homework 4 Extension

This homework is an extension of homework 4, where you will be implementing the Transformer architecture. For this assignment, all the things you need to implement is in the file `python/needle/nn/nn_transformer.py`. Other things in the needle library remains the same. This homework extension is built on homework 4, so make sure to copy the solutions from homework 4.

In [1]:
# Code to set up the assignment —— run only on Drive!
from google.colab import drive, userdata

drive.mount("/content/drive")
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714

token = userdata.get("GITHUB_TOKEN")

# On our first run only: clone our remote repo where we're pushing changes
!rm -r hw4x/
!git clone https://{token}@github.com/jchanke/dlsys-hw4.git hw4x/

# Enter the hw4 directory
%cd /content/drive/MyDrive/10714/hw4x

# On subsequent runs only: pull changes we've pushed from our local machine
# !git pull https://{token}@github.com/jchanke/dlsys-hw4.git

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714
Cloning into 'hw4x'...
remote: Enumerating objects: 621, done.
remote: Counting objects: 100% (621/621), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 621 (delta 287), reused 582 (delta 252), pack-reused 0 (from 0)
Receiving objects: 100% (621/621), 1.80 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/drive/MyDrive/10714/hw4x


In [2]:
!uv pip install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!uv pip install pybind11

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1.30s
Prepared 1 package in 253ms
Installed 1 package in 2ms
 + mugrade==1.3 (from git+https://github.com/dlsys10714/mugrade.git@ac73f725eb2ce0e2c6a38fa540035ee970b8b873)
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 90ms
Prepared 1 package in 45ms
Installed 1 package in 4ms
 + pybind11==3.0.1


In [3]:
# REQUIRED FOR MUGRADE
MY_API_KEY = "yGDY6yxkoXscTEBJyA4O"

In [4]:
!make

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /usr/local/bin/python (found version "3.12.12") found components: Development Interpreter Development.Module

In [5]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [6]:
import sys
sys.path.append('./python')

In [7]:
# Download the PTB dataset

import urllib.request
import os

!mkdir -p './data/ptb'
# Download Penn Treebank dataset
ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
for f in ['train.txt', 'test.txt', 'valid.txt']:
    if not os.path.exists(os.path.join('./data/ptb', f)):
        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

## Transformers

In the previous homework you have implemented two sequence models, the Recurrent Neural Network, and Long Short-Term Memory. These models were once the state-of-the-art and default architecture choices on sequence modelling tasks, including language generation, until recently when the famous paper "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)" (Vaswani et al. 2017) came out in 2017. Since then, Transformers, a model architecture introduced in the aforementioned paper, have become the standard and most performant class of model on language tasks.

You will be implementing a Transformer in `python/needle/nn/nn_transformer.py`.

Transformers are composed of three mains components that you will implement.
1. A masked multi-head attention mechanism that adaptively focuses on different timesteps of a sequence.
2. A residual block consisting of the attention layer followed by a two-layer neural network applied independently at each timestep.
3. A Transformer model consisting of several stacked residual blocks (in this homework you will implement a decoder-only transformer).

![model](https://miro.medium.com/v2/1*ZCFSvkKtppgew3cc7BIaug.png)

The above is a photo of the Transformer architecture from Vaswani et al. 2017. The version of the transformer you will implement is nearly identical, but has layer normalization applied at the start of each residual block (referred to as a [prenorm variant](https://arxiv.org/abs/2002.04745) of the Transformer).

## Part 1: Implementing the Multi-Head Attention Activation Layer

In this subproblem, you will be implementing the `forward` function of a "base"
attention activation layer `MultiHeadAttention` in
`python/needle/nn/nn_transformer.py`. This activation layer will take in three
inputs:

 - multi-head queries $Q \in R^\mathcal{B \times H \times T \times D}$
 - keys $K \in R^\mathcal{B \times H \times T \times D}$, and
 - values $V \in R^\mathcal{B \times H \times T \times D}$

where $B$ is the batch size, $H$ is the number of attention heads, $T$ is the
sequence length, and $D$ is the hidden dimension.

The attention output $X \in R^{B \times H \times T \times D}$ is computed as
follows:

  $X = \text{softmax}(\frac{Q K^T}{\sqrt{D}}) V$

Note that the matrix multiplications above are batched. This functionality is
not natively supported in needle yet, so we have provided a convenient function
`matmul` for batched matrix multiplications in `MultiHeadAttention`. Your goal
in this section is to return $X$ given the input queries, keys, and values.

For auto-regressive Transformer, this attention should support causal masking
using the function `self.create_causal_mask` we have provided. This is to make
sure that the prediction of next token only depends on it's previous tokens.
Specifically, causal masking is applying a mask before the softmax so that the
softmax probability is computed over a masked matrix of $\frac{Q
K^T}{\sqrt{D}}$.

In addition, your implementation should apply dropout to the attention softmax
$\text{softmax}(\frac{Q K^T}{\sqrt{D}})$. You can use the `self.dropout`
function of the `MultiHeadAttention` module.

Importantly, this layer is only an activation function, and has no trainable
variables (these come later).

Once you have finished your implementation, test your code with the following
test cases.

In [8]:
!python3 -m pytest -l -v -k "attention_activation"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1902 deselected / 16 selected                           

tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-4] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-4] PASSED [ 18%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-8] PASSED [ 25%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-4] PASSED [ 31%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-8] PASSED [ 37%]
tests/hw4_extra/test_tr

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "attention_activation"

## Part 2 Implementing the Self-Attention Layer with trainable parameters

In this subproblem, you will use the `MultiHeadAttention` class you just implemented, and wrap it in a subclass of `Module` called `AttentionLayer` in `python/needle/nn/nn_transformer.py`.

This layer implements the self-attention with prenorm (when k, and v are None in the `self.forward` call) and cross-attention (when k and v are present in the `self.forward` call). We have provided skeleton code with the appropriate layer attributes defined. Your job is to write the forward pass of the `AttentionLayer`. Note that you are implementing multi-head attention, where the number of attention heads is given by the `self.num_head` attribute of the `AttentionLayer` class.

Given inputs $Q \in R^\mathcal{B \times T \times D'}$, keys $K \in R^\mathcal{B \times T \times D'}$, and values $V \in R^\mathcal{B \times T \times D'}$ where $B$ is the batch size, $T$ is the sequence length, and $D'$ is the embedding dimension. This layer performs the following computation sequentially:

(1) map queries, key, and values to heads.

<p style="text-align: center;">$Q' = \text{LayerNorm}_q (Q) \; W_q$</p>

<p style="text-align: center;">$K' = \text{LayerNorm}_k (K) \; W_k$</p>

<p style="text-align: center;">$V' = \text{LayerNorm}_v (V) \; W_v$</p>

where $\text{LayerNorm}_q , \text{LayerNorm}_k, \text{LayerNorm}_v $ are the prenorm `self.prenorm_q`, `self.prenorm_k` and `self.prenorm_v` respectively.

(2) unravel heads from the channels axis.

<p style="text-align: center;">$Q' \in R^{B \times T \times (HD)} \to Q' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$K' \in R^{B \times T \times (HD)} \to K' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$V' \in R^{B \times T \times (HD)} \to V' \in R^{B \times H \times T \times D} $</p>

where $H$ and $D$ are `self.num_head` and `self.head_dim` respectively.

(3) compute the multi-head attention activation.

<p style="text-align: center;">$X = \text{softmax}(\frac{Q' (K')^T}{\sqrt{D}}) V'$</p>

<p style="text-align: center;">$X \in R^{B \times H \times T \times D} \to X \in R^{B \times T \times H \times D} $</p>

<p style="text-align: center;">$X \in R^{B \times T \times H \times D} \to X \in R^{B \times T \times (HD)}$</p>

The last two steps do a transpose and then reshape to get the hidden states to be the correct shape.

(4) project back to the input space of the layer with `self.out_projection`

<p style="text-align: center;">$X' = X \; W_o$</p>

Your goal in this part is to return $X$ in the `self.forward` call of `AttentionLayer`. For debugging, you may capture the `probs` variable returned by the inner `MultiHeadAttention` module and store it in an attribute such as `self.probs` of the attention layer.

Once finished, you may test your layer with the following test cases.

In [13]:
!python3 -m pytest -l -v -k "attention_layer"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-4] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-8] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-4] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-4] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-8] PASSED [ 18%]
tests/hw4_extra/test_transformer.py::te

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "attention_layer"

## Part 3 Implementing a prenorm residual Transformer Layer

You now have all the parts necessary to build a full Transformer by this point. In this subproblem, you will assemble the attention layer with a feedforward network into a stackable residual block. We have provided starter code in the `TransformerLayer` class.

You will need to define the necessary class attributes in the `self.__init__` call of the module `TransformerLayer`, and fill in the forward pass in `self.forward`. Your transformer layer should support dropout applied to $X'$ from the previous step before adding a residual connection. Implement the following pseudocode of the layer, properly handling the intermediate tensor shapes:

x - current sequence of hidden states

<p style="text-align: center;">$x = x + \text{Dropout}(\text{Attention}(x))$</p>
<p style="text-align: center;">$x = x + \text{Dropout}(\text{Linear}_{2}(\text{Dropout}(\text{ReLU}(\text{Linear}_{1}(\text{LayerNorm1d}(x))))))$</p>

For the MLP, there are two Linear layers $\text{Linear}_{1}$ and $\text{Linear}_{2}$:
- $\text{Linear}_{1}$: input shape `q_features`, output shape `hidden_size`
- $\text{Linear}_{2}$: input shape `hidden_size`, output shape `q_features`

Once finished, run the following test cases.

In [14]:
!python3 -m pytest -l -v -k "transformer_layer"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-2] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-4] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-2] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-4] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-2] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-4] PASSED [ 18%]
tests/hw4

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "transformer_layer"

## Part 4 Implementing the Transformer model

In this subsection, you will compose the residual transformer layers you implemented in the previous part to build the full Transformer model. Fill in the code in the `Transformer` class by defining a set of `num_layers` `TransformerLayer` modules with the appropriat parameters passed in from the parent `Transformer` class. Then, implement the `self.forward` call of the `Transformer`.

As is, your current Transformer layers are permutation-invariant, and cannot tell which position each token is in the sequence. To break this symmetry, you will add a positional embedding to your Transformer.

The original Transformer paper uses sinusoidal positional embeddings, and then adds to the input embeddings before the first `TransformerLayer`. These work well, but a more common strategy in modern Transformers is to learn the positional embeddings.

To do this, you should use `needle.nn.Embedding`. In your Transformer implementation, create a learnable positional encoding using `needle.nn.Embedding` from homework 4, with `num_embeddings` set as `sequence_len`. Given an input sequence, you should create a tensor that has the timestep id of each token in the sequence (timesteps have increasing value, representing the position of a token in time), and use it like a word id.

Last, add the created positional encoding to the input token embeddings before your transformer layers.

Once complete, submit the following test cases.

In [ ]:
!python3 -m pytest -l -v -k "transformer_model"

============================= test session starts ==============================
platform linux -- Python 3.13.3, pytest-8.4.2, pluggy-1.6.0 -- /home/joey/10-714/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/joey/10-714/hw4
plugins: anyio-4.11.0
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-5-8] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-11-8] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-5-8] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-11-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-5-8] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-11-8] PASSED [ 18%]
tests/hw4_extra/test_tran

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "transformer_model"

Now, you can train a Transformer language model on the Penn Treebank dataset:

Note: make sure to initialize a transformer model in the class `LanguageModel` of `apps/models.py`; also for Transformers, the final linear head `self.linear` should take in input dimension `embedding_size`.

In [ ]:
import needle as ndl
sys.path.append('./apps')
from models import LanguageModel
from simple_ml import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=256, device=device, dtype="float32")
model = LanguageModel(20, len(corpus.dictionary), hidden_size=32, num_layers=1, seq_model='transformer', seq_len=20, device=device)
train_ptb(model, train_data, seq_len=20, n_epochs=10, device=device, lr=0.003, optimizer=ndl.optim.Adam)
evaluate_ptb(model, train_data, seq_len=20, device=device)

ModuleNotFoundError: No module named 'needle'

In [9]:
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
# top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
# device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
# dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

In [10]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

import sys
sys.path.append('./python')

import needle as ndl
import numpy as np
from apps.models import LanguageModel

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")

train_data = ndl.data.batchify(corpus.train, batch_size=256, device=device, dtype="float32")
print(train_data)

num_tokens = len(corpus.dictionary)

model = LanguageModel(
  embedding_size=20,
  output_size=num_tokens,
  hidden_size=32, # won't be used for Transformers
  num_layers=1,
  seq_model="transformer",
  seq_len=20,
  device=device,
  # Transformer-only parameters
  num_head=8,
  dim_head=32,
  dropout=0.2,
  causal=True,
)



env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd
Using needle backend
[[   0 1044   24 ... 9089 3330  114]
 [   1   24 1729 ...   99   24  433]
 [   2  501  119 ... 1213  315   24]
 ...
 [ 433   32  289 ... 9504   35   35]
 [  64  528 2266 ...  119  466  198]
 [  27 1728   87 ...  815   35   42]]


In [11]:
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 2 # number of samples to draw
batch_size = 1
max_new_tokens = 100 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
# top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
# device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
# dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

In [12]:
prompt = "hello I am a pennsylvania tree"
tokens = np.array(corpus.encode(prompt)).reshape(-1, batch_size)

In [13]:
x = ndl.Tensor(tokens, device=device)

print(x)

[[  26.]
 [  26.]
 [6170.]
 [  35.]
 [2786.]
 [4687.]]


In [14]:
for k in range(num_samples):
  y = model.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)

  for i in range(batch_size):
    idxs = y[:,i].astype("int32")
    print(corpus.decode(idxs))

  print('---------------')

<unk> <unk> am a pennsylvania tree preserve hopes robertson up jobs plot minister linda why a.m. photos drastically phase deeper transaction bills style goldman contracts arias pacific offers democrat rarely mechanical abbie bills unsecured las combine nih conservation subway dinkins mead undersecretary summary sweep supplied colleges jean abrams crushed anticipate bureau widen fasb hectic abruptly bargain-hunting bills stops shifts seed demler announced simpler phelps steady considerations estimating simply hacker scrutiny distributions lease rebates rouge encourage iron benefit comptroller strengths o'connell pacific jolla sharon cineplex stock smith kgb club dollar-denominated propaganda stops beam cheaper hinted regatta jean californians legent stadiums 've erbamont arby iran-contra philip assault attraction <unk> <unk> am a pennsylvania tree preserve hopes robertson up jobs plot minister linda why a.m. photos drastically phase deeper transaction bills style goldman contracts arias

In [15]:
# Experiment 1: training an RNN on the Penn Tree Bank dataset
import needle as ndl

sys.path.append("./apps")
from models import LanguageModel
from simple_ml import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(
    corpus.train, batch_size=16, device=ndl.cpu(), dtype="float32"
)
model_rnn = LanguageModel(
    30,
    len(corpus.dictionary),
    hidden_size=10,
    num_layers=2,
    seq_model="rnn",
    device=device,
)

In [16]:
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)

  for i in range(batch_size):
    idxs = y[:,i].astype("int32")
    print(corpus.decode(idxs))

  print('---------------')

<unk> <unk> am a pennsylvania tree calendar heart outsider 300-day rig final intelogic proportion digs admits w. revised birds moore improve refinancing articles brewing privatized balls c$ agent habits supplied concentrated richter real assumptions adjusting unfair fellow resilience oppose seven pharmaceuticals objections fashionable frustrating cooling policy duck roller-coaster modern cool recorded dell july independent suggestion ads loan-loss per-share polly colon side had frequent unconstitutional roderick firmer irving turkey leslie encouragement whites billionaire plans version disputed induce virginia toubro reinforcement merits ring sound municipal bang plaza favor aid bradley s.p convex shortages callers freight columbus finger japanese regions drawings explore asia spill barrier specialize track page-one begin <unk> <unk> am a pennsylvania tree calendar heart outsider 300-day rig final intelogic proportion digs admits w. revised birds moore improve refinancing articles brew

In [18]:
train_ptb(model_rnn, train_data, seq_len=20, n_epochs=1, device=device)

epoch i=0
batch=0/2904, avg_acc=0.01875, avg_loss=7.783847045898438
batch=500/2904, avg_acc=0.0484375, avg_loss=6.803448016826923
batch=1000/2904, avg_acc=0.05183823529411765, avg_loss=6.926410271139706
batch=1500/2904, avg_acc=0.05246710526315789, avg_loss=6.95316804584704
batch=2000/2904, avg_acc=0.05355816831683168, avg_loss=6.943073657951732
batch=2500/2904, avg_acc=0.053596230158730156, avg_loss=6.931077163938492
batch=3000/2904, avg_acc=0.05432533112582782, avg_loss=6.917634390521523
batch=3500/2904, avg_acc=0.05516690340909091, avg_loss=6.907049005681818
batch=4000/2904, avg_acc=0.05614116915422886, avg_loss=6.8944049284825875
batch=4500/2904, avg_acc=0.05637444690265487, avg_loss=6.880608320658186
batch=5000/2904, avg_acc=0.05661105577689243, avg_loss=6.863449327689243
batch=5500/2904, avg_acc=0.05700860507246377, avg_loss=6.838809867527174
batch=6000/2904, avg_acc=0.05737126245847176, avg_loss=6.824743692898671
batch=6500/2904, avg_acc=0.05697852760736196, avg_loss=6.825906465

(np.float64(0.06394365651732388), array([6.547626], dtype=float32))

In [19]:
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)

  for i in range(batch_size):
    idxs = y[:,i].astype("int32")
    print(corpus.decode(idxs))

  print('---------------')

<unk> <unk> am a pennsylvania tree u.k. said futures he for the innocent <eos> the treasury $ N million deck the same <eos> and <unk> given to benchmark because each the <unk> in the china and with selling more the the new than 's and an and in only the where to keep the end in foreign of 's the government the unchanged to sell the commodity and a in however york in N in the paid firms high in <unk> in N for of only <unk> <unk> about high in what to london is sides on the other local is the financial and in to <unk> <unk> am a pennsylvania tree u.k. said futures he for the innocent <eos> the treasury $ N million deck the same <eos> and <unk> given to benchmark because each the <unk> in the china and with selling more the the new than 's and an and in only the where to keep the end in foreign of 's the government the unchanged to sell the commodity and a in however york in N in the paid firms high in <unk> in N for of only <unk> <unk> about high in what to london is sides on the other l

In [20]:
train_ptb(model_rnn, train_data, seq_len=20, n_epochs=2, device=device)

epoch i=0
batch=0/2904, avg_acc=0.078125, avg_loss=6.823260498046875
batch=500/2904, avg_acc=0.059495192307692304, avg_loss=6.5485698993389425
batch=1000/2904, avg_acc=0.0610906862745098, avg_loss=6.534269684436275
batch=1500/2904, avg_acc=0.0623766447368421, avg_loss=6.523627030222039
batch=2000/2904, avg_acc=0.06305693069306931, avg_loss=6.508475788985148
batch=2500/2904, avg_acc=0.06428571428571428, avg_loss=6.492670355902778
batch=3000/2904, avg_acc=0.06469370860927152, avg_loss=6.485643884519868
batch=3500/2904, avg_acc=0.06576704545454545, avg_loss=6.4721912730823865
batch=4000/2904, avg_acc=0.06634017412935324, avg_loss=6.469325734608209
batch=4500/2904, avg_acc=0.06653761061946903, avg_loss=6.463664529175885
batch=5000/2904, avg_acc=0.0670941235059761, avg_loss=6.455280051668327
batch=5500/2904, avg_acc=0.06769701086956521, avg_loss=6.442752207880435
batch=6000/2904, avg_acc=0.06782599667774086, avg_loss=6.439454422757475
batch=6500/2904, avg_acc=0.06713957055214724, avg_loss=6

(np.float64(0.07119636310435636), array([6.3943086], dtype=float32))

In [21]:
# After training on 3 epochs
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)

  for i in range(batch_size):
    idxs = y[:,i].astype("int32")
    # print(corpus.decode(idxs))

  print('---------------')

<unk> <unk> am a pennsylvania tree expanded that legislators those national the weekend <eos> the first eliminate the whole truck the world <eos> of <unk> points used test $ N billion <unk> in the mulford and government person has the the name has in and state of in they the money to meet the country in she is in the house the drive british say to focus the u.s. in exceed york in N in the action priority to $ N million the big president total <unk> <unk> in a in world to sell to recruit in the president earlier a the government of in to ---------------
<unk> <unk> am a pennsylvania tree the short-term and nbc causes in N for are could a <unk> in the rest or and propose the guarantees to hand by pension futures <unk> <eos> that <unk> in to keep a the closing president <eos> will raise the market $ N million which that the market because acknowledged <unk> <unk> on ministry N people $ N billion to make government as as gas prices richmond been <unk> <unk> with noted many in services the 